In [0]:
##################################
# Author: Christopher Audretsch
# Date: April 27, 2020
# Class: Image Processing, Adeel Bhutta
# This is the generic image classification network, based on Wang et al's paper.
##################################
#Import Libraries
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from os import listdir
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import random

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
args={}
kwargs={}
args['batch_size']=1024
args['test_batch_size']=1024
args['epochs']=100  #The number of Epochs is the number of times you go through the full dataset. 
args['lr']=0.003 #Learning rate is how fast it will decend. 
args['momentum']=0.5 #SGD momentum (default: 0.5) Momentum is a moving average of our gradients (helps to keep direction).

args['seed']=1 #random seed
args['log_interval']=10
args['cuda']=True

path = "/content/drive/My Drive/Colab Notebooks/UTKFace"

In [0]:
class EthnicityDataset(Dataset):
    def __init__(self, root_dir, transform = None, train = True, post = False):
        # root_dir (string): Directory with all the images.
        # transform: 
        random.seed(42)
        self.root_dir = root_dir
        self.image_paths = listdir(root_dir)
        random.shuffle(self.image_paths)
        if post:
          self.image_paths = self.image_paths[12000:]
        elif train:
          self.image_paths = self.image_paths[:10000]
        else:
          self.image_paths = self.image_paths[10000:12000]
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.root_dir + '/' + self.image_paths[idx])
        # The labels of each face image is embedded in the file name, formated like [age]_[gender]_[race]_[date&time].jpg
        # race is an integer from 0 to 4, denoting White, Black, Asian, Indian, and Others
        race = (self.image_paths[idx].split('_')[2])
        if race != '0' and race != '1' and race != '2' and race != '3' and race != '4':
          race = '4'
        race = int(race)
        if self.transform:
          image = self.transform(image)
        return image, race

In [0]:
ds_train = EthnicityDataset(path,transform = transforms.Compose([
                       transforms.Grayscale(),
                       transforms.Resize(64),
                       transforms.ToTensor(),
                       transforms.Normalize((.497,), (.226,))
                   ]))
ds_test = EthnicityDataset(path, train = False, transform = transforms.Compose([
                       transforms.Grayscale(),
                       transforms.Resize(64),
                       transforms.ToTensor(),
                       transforms.Normalize((.497,), (.226,))
                   ]))

In [0]:
#load the data
train_loader = torch.utils.data.DataLoader(ds_train, batch_size=args['batch_size'], num_workers = 4, pin_memory = True, shuffle=True, **kwargs)
#test_loader = torch.utils.data.DataLoader(ds_train2, batch_size=args['batch_size'], num_workers = 4, pin_memory = True, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(ds_test,batch_size=args['test_batch_size'], num_workers = 4, pin_memory = True, shuffle=True, **kwargs)

In [0]:
class Net(nn.Module):
    #This defines the structure of the NN.
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5,padding=2,stride=1)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=5,padding=2,stride=1)
        self.conv3 = nn.Conv2d(32,64, kernel_size=5,padding=2,stride=1)
        self.fc1 = nn.Linear(3136, 128)
        self.fc2 = nn.Linear(128, 5)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size=3,stride=2))
        x = F.relu(F.avg_pool2d(self.conv2(x), kernel_size=3,stride=2))
        x = F.relu(F.avg_pool2d(self.conv3(x), kernel_size=3,stride=2))
        #Fully Connected Layer/Activation
        x = x.view(-1, 3136)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)


In [0]:
losses = []
tests = []
def train(epoch):
    model.train()
    #print('before for loop')
    for batch_idx, (data, target) in enumerate(train_loader):
        if args['cuda']:
            data, target = data.cuda(), target.cuda()
        #Variables in Pytorch are differenciable. 
        data, target = Variable(data), Variable(target)
        #This will zero out the gradients for this batch. 
        optimizer.zero_grad()
        output = model(data)
        # Calculate the loss The negative log likelihood loss. It is useful to train a classification problem with C classes.
        loss = F.nll_loss(output, target)
        #dloss/dx for every Variable 
        loss.backward()
        #to do a one-step update on our parameter.
        optimizer.step()
        #Print out the loss periodically. 
        if batch_idx % args['log_interval'] == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
    losses.append(loss.item())

def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if args['cuda']:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.nll_loss(output, target, size_average=False).item() # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        x = pred.eq(target.data.view_as(pred)).long().cpu().sum()
        correct += x
    tests.append(int(correct) / len(test_loader.dataset))
    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [0]:
#model = Net()
if args['cuda']:
    model.cuda()

optimizer = optim.SGD(model.parameters(), lr=args['lr'], momentum=args['momentum'])

#for epoch in range(1, args['epochs'] + 1):
for epoch in range(1, 50):
    train(epoch)
    test()


Train Epoch: 1 [0/10000 (0%)]	Loss: 0.167846
Train Epoch: 1 [2560/10000 (25%)]	Loss: 0.196985
Train Epoch: 1 [5120/10000 (50%)]	Loss: 0.144907
Train Epoch: 1 [7680/10000 (75%)]	Loss: 0.150698


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:34: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Average loss: 1.2129, Accuracy: 1476/2000 (74%)

Train Epoch: 2 [0/10000 (0%)]	Loss: 0.163362
Train Epoch: 2 [2560/10000 (25%)]	Loss: 0.158608
Train Epoch: 2 [5120/10000 (50%)]	Loss: 0.161188
Train Epoch: 2 [7680/10000 (75%)]	Loss: 0.184433

Test set: Average loss: 1.2114, Accuracy: 1479/2000 (74%)

Train Epoch: 3 [0/10000 (0%)]	Loss: 0.144052
Train Epoch: 3 [2560/10000 (25%)]	Loss: 0.178970
Train Epoch: 3 [5120/10000 (50%)]	Loss: 0.148086
Train Epoch: 3 [7680/10000 (75%)]	Loss: 0.194301

Test set: Average loss: 1.2492, Accuracy: 1434/2000 (72%)

Train Epoch: 4 [0/10000 (0%)]	Loss: 0.313484
Train Epoch: 4 [2560/10000 (25%)]	Loss: 0.133979
Train Epoch: 4 [5120/10000 (50%)]	Loss: 0.198121
Train Epoch: 4 [7680/10000 (75%)]	Loss: 0.180826

Test set: Average loss: 1.1825, Accuracy: 1465/2000 (73%)

Train Epoch: 5 [0/10000 (0%)]	Loss: 0.213329
Train Epoch: 5 [2560/10000 (25%)]	Loss: 0.118357
Train Epoch: 5 [5120/10000 (50%)]	Loss: 0.294744
Train Epoch: 5 [7680/10000 (75%)]	Loss: 0

In [0]:
plt.plot(tests)
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.title('Model Accuracy Over Time')

NameError: ignored

In [0]:
model

NameError: ignored